# Sequence to Sequence Learning with Neural Networks
- reference code: https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### tokenizer를 위한 spacy 다운 (영어: en / de: 독일어)

In [3]:
!python -m spacy download en
!python -m spacy download de

     |████████████████████████████████| 12.0 MB 534 kB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/conda/lib/python3.7/site-packages/en_core_web_sm -->
/opt/conda/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 7.9 MB/s eta 0:00:01    |██████▍                         | 3.0 MB 7.9 MB/s eta 0:00:02
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.3.0-py3-none-any.whl size=14907581 sha256=5e8f2f9eebaf4f44b804e7bec8ef434ee0dd4c83f3cdcdc7f6e53d0558b9cded
  Stored in directory: /tmp/pip-ephem-wheel-cache-7d8067ds/wheels/75/30/c3/ea1c6002eede7f49c8ab017ce62a2981a87b1cd39fab6e6a65
Successfully built de-core-news-sm


✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/opt/conda/lib/python3.7/site-packages/de_core_news_sm -->
/opt/conda/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [4]:
spacy_en = spacy.load('en')
spacy_de = spacy.load('de')

In [5]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::]

### example for tokenizer
-> reverse!

In [6]:
# source # reverse
tokenize_en('Hello. My name is Karlie.')

['.', 'Karlie', 'is', 'name', 'My', '.', 'Hello']

In [7]:
# target
tokenize_de("Hallo. Ich heiße Carly.") 

['Hallo', '.', 'Ich', 'heiße', 'Carly', '.']

In [9]:
SRC = Field(tokenize = tokenize_en,
           init_token = '<sos>',
           eos_token = '<eos>',
            unk_token = '<unk>',
           lower = True)

TRG = Field(tokenize = tokenize_de,
           init_token = '<sos>',
           eos_token = '<eos>',
             unk_token = '<unk>',
           lower = True)

/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Download the data (Multi30k)

In [10]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.en', '.de'),                                                    
                                                   fields = (SRC, TRG))
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 892kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 173kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 165kB/s]
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [11]:
print(vars(train_data.examples[0]))

{'src': ['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two'], 'trg': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']}


In [12]:
SRC.build_vocab(train_data, max_size = 16000)
TRG.build_vocab(train_data, max_size = 8000)

In [13]:
# change pad and unk id for packed sequence
SRC.vocab.stoi['<unk>'] = 1
SRC.vocab.stoi['<pad>'] = 0

In [14]:
# vocab lists
[val for val in SRC.vocab.stoi.items()][:10]

[('<unk>', 1),
 ('<pad>', 0),
 ('<sos>', 2),
 ('<eos>', 3),
 ('a', 4),
 ('.', 5),
 ('in', 6),
 ('the', 7),
 ('on', 8),
 ('man', 9)]

In [15]:
TRG.vocab.stoi['<unk>'] = 1
TRG.vocab.stoi['<pad>'] = 0

In [16]:
# vocab lists
[val for val in TRG.vocab.stoi.items()][:10]

[('<unk>', 1),
 ('<pad>', 0),
 ('<sos>', 2),
 ('<eos>', 3),
 ('.', 4),
 ('ein', 5),
 ('einem', 6),
 ('in', 7),
 ('eine', 8),
 (',', 9)]

In [ ]:
print(f"src vocab size: {len(SRC.vocab)}")
print(f"trg vocab size: {len(TRG.vocab)}")

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [18]:
BATCH_SIZE = 3
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size = BATCH_SIZE, device = device,
    sort_key=lambda x:len(x.src), # src 문장을 길이로 sort,
#     sort_within_batch = True # 이걸 하는게 의미가 있나요?-> 없었당
)

/opt/conda/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [19]:
print(f"number of train batch: {len(train_iterator)}")
print(f"number of valid batch: {len(valid_iterator)}")
print(f"number of test batch: {len(test_iterator)}")

number of train batch: 9667
number of valid batch: 338
number of test batch: 334


In [20]:
## data check
cnt = 0
for a , b in train_iterator:
    print('source text')
    print(a[0].shape)
    print(a[0])
#     print(a.size())
    print()
    print('target text')
    print(a[1].shape)
    print(a[1])
    cnt += 1
    print('********************')
    if cnt == 2:
        break

/opt/conda/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


source text
torch.Size([14, 3])
tensor([[   2,    2,    2],
        [   5,  229,    5],
        [  82,   33,  186],
        [  46,   82,  200],
        [  13,   28,  456],
        [ 828, 4144,  266],
        [  17,   42, 6257],
        [  19,  607,    7],
        [  12, 3373,    3],
        [  38,   27,    0],
        [   4,  940,    0],
        [   3,  665,    0],
        [   0,    4,    0],
        [   0,    3,    0]], device='cuda:0')

target text
torch.Size([16, 3])
tensor([[   2,    2,    2],
        [  76,    5,   17],
        [  54,  572,    0],
        [ 185,  303, 3817],
        [  23,   39, 4058],
        [   4, 4647,  230],
        [   3,  197,  402],
        [   0,  787,   12],
        [   0,    9,    4],
        [   0,   35,    3],
        [   0,    5,    0],
        [   0,  610,    0],
        [   0,   25,    0],
        [   0,  441,    0],
        [   0,    4,    0],
        [   0,    3,    0]], device='cuda:0')
********************
source text
torch.Size([27, 3])
tensor

In [21]:
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size = BATCH_SIZE, device = device,
    sort_key=lambda x:len(x.src), # src 문장을 길이로 sort,
#     sort_within_batch = True # 이걸 하는게 의미가 있나요?-> 없었당
)

## model

### encoder
- return: 
1. outputs: the top-layer hidden state for each time-step
2. hidden: the final hidden state for each layer, $h_T$, stacked on top of each other
3. cell: the final cell state for each layer, $c_T$, stacked on top of each other

## packed sequence ?

Holds the data and list of batch_sizes of a packed sequence.

All RNN modules accept packed sequences as inputs.

ref: https://simonjisu.github.io/nlp/2018/07/05/packedsequence.html

결국 pad처리를 보다 비용을 효율적으로 하기 위해, pad 갯수 순으로 정렬함.

In [22]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, src):
#         print('@@@ encoder')
        
        src_ = src.transpose(1, 0)

        # for packed sequence: for cost saving
        # src_.shape = (batch, seq len)
        input_lengths = torch.LongTensor([torch.max(src_[i, :].data.nonzero())+1 for i in range(src_.size(0))])
        input_lengths, sorted_idx = input_lengths.sort(0, descending=True)
        input_seq2idx = src_[sorted_idx]

        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        
        embedded = embedded.transpose(1, 0)
        # embedded = [batch size,src len, emb dim]
        # Pack padded batch of sequences for RNN module
        packed_input = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.tolist(), batch_first=True)
        
        packed_output, (hidden, cell) = self.rnn(packed_input)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        
        #outputs are always from the top hidden layer
#         print('hidden', hidden.shape)
#         print('cell', cell.shape)
        # output.shape = (batch size, src len, hid dim * n directions)
        output = output.transpose(1, 0)
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        return output, hidden, cell

### encoder output size check
hidden torch.Size([2, 128, 512])
cell torch.Size([2, 128, 512])

### decoder

encoder랑 구조는 같은데, input이 length 1로 들어가게 하는 거랑 prediction값을 dim = 1로 축소하는 거만 다름

In [23]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        if input.dim() == 2:
            input = input
        else:
            input = input.unsqueeze(0)
        
        #input = [1, batch size] # 1: 문장 길이
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
#         output = F.log_softmax(prediction, dim=1)
#         print('output', output.shape) #output torch.Size([128, 8004])
        return prediction, hidden, cell

In [52]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

## Seq2Seq
For the final part of the implemenetation, we'll implement the seq2seq model. This will handle:

- receiving the input/source sentence
- the encoder to produce the context vectors
- the decoder to produce the predicted output/target sentence

### teacher forcing

In [53]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    #src, trg, teacher_forcing_ratio = 0.5):
    def forward(self,src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_outputs, hidden, cell = self.encoder(src)
        

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
#         return beam_decode(input, hidden, cell)
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs
            

In [54]:
model = Seq2Seq(enc, dec, device).to(device)

In [55]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9799, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8004, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=8004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [56]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,020,036 trainable parameters


In [111]:
optimizer = optim.SGD(model.parameters(), lr = 0.7, momentum=0) # 논문 구현 방식. 근데 성능은 별로
optimizer_better = optim.Adam(model.parameters()) # adam이 짱임

In [112]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [59]:
TRG.pad_token, TRG_PAD_IDX

('<pad>', 0)

In [60]:
def train(model, iterator, optimizer, criterion, clip,epoch):
    model.train()
    epoch_loss = 0
    
    iter_check = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # trg = [(trg len - 1) * batch size]
        # output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
               
        # custom scheduler
        gamma = 0.5      
        if epoch + 1 > 5 and i % (len(iterator) // 2) == 0 and i != 0:
            for group in optimizer.param_groups:
                group['lr'] = group['lr'] * gamma
                iter_check += 1
        
                print(iter_check,group['lr'] )

                if epoch + 1 == 7 and iter_check == 1:
                    optimizer.step()
                    epoch_loss += loss.item()
                    return epoch_loss / len(iterator),iter_check
        optimizer.step()
        epoch_loss += loss.item()
        
    print('Epoch-{0} lr: {1}'.format(epoch+ 1, optimizer.param_groups[0]['lr']))
    return epoch_loss / len(iterator),iter_check

In [115]:
def train_better(model, iterator, optimizer, criterion, clip,epoch):
    model.train()
    epoch_loss = 0
    
    iter_check = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # trg = [(trg len - 1) * batch size]
        # output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
               
       
        optimizer.step()
        epoch_loss += loss.item()
        
    print('Epoch-{0} lr: {1}'.format(epoch+ 1, optimizer.param_groups[0]['lr']))
    return epoch_loss / len(iterator),iter_check

In [61]:
def evaluate(model, iterator, criterion):    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [62]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

논문 구현 방식 (halving lr at every 0.5 epoch )

In [63]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, iter_check = train(model, train_iterator, optimizer, criterion, CLIP,epoch)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # epoch 7.5 에 save
    if epoch+1 == 7 and iter_check == 1:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        print('model save..!')
        break
        
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch-1 lr: 0.7
Epoch: 01 | Time: 0m 50s
	Train Loss: 5.776 | Train PPL: 322.444
	 Val. Loss: 5.175 |  Val. PPL: 176.790
Epoch-2 lr: 0.7
Epoch: 02 | Time: 0m 49s
	Train Loss: 5.235 | Train PPL: 187.752
	 Val. Loss: 5.122 |  Val. PPL: 167.590
Epoch-3 lr: 0.7
Epoch: 03 | Time: 0m 50s
	Train Loss: 5.139 | Train PPL: 170.486
	 Val. Loss: 5.134 |  Val. PPL: 169.649
Epoch-4 lr: 0.7
Epoch: 04 | Time: 0m 49s
	Train Loss: 5.084 | Train PPL: 161.455
	 Val. Loss: 5.233 |  Val. PPL: 187.324
Epoch-5 lr: 0.7
Epoch: 05 | Time: 0m 49s
	Train Loss: 5.054 | Train PPL: 156.671
	 Val. Loss: 4.982 |  Val. PPL: 145.768
1 0.35
2 0.175
Epoch-6 lr: 0.175
Epoch: 06 | Time: 0m 50s
	Train Loss: 4.992 | Train PPL: 147.205
	 Val. Loss: 4.943 |  Val. PPL: 140.244
1 0.0875
model save..!


잘 나오는 방식

In [116]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, iter_check = train_better(model, train_iterator, optimizer_better, criterion, CLIP,epoch)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

/opt/conda/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch-1 lr: 0.001
Epoch: 01 | Time: 0m 50s
	Train Loss: 3.975 | Train PPL:  53.243
	 Val. Loss: 4.232 |  Val. PPL:  68.852
Epoch-2 lr: 0.001
Epoch: 02 | Time: 0m 49s
	Train Loss: 3.712 | Train PPL:  40.922
	 Val. Loss: 4.055 |  Val. PPL:  57.689
Epoch-3 lr: 0.001
Epoch: 03 | Time: 0m 49s
	Train Loss: 3.495 | Train PPL:  32.964
	 Val. Loss: 3.855 |  Val. PPL:  47.235
Epoch-4 lr: 0.001
Epoch: 04 | Time: 0m 49s
	Train Loss: 3.309 | Train PPL:  27.370
	 Val. Loss: 3.725 |  Val. PPL:  41.490
Epoch-5 lr: 0.001
Epoch: 05 | Time: 0m 49s
	Train Loss: 3.153 | Train PPL:  23.415
	 Val. Loss: 3.617 |  Val. PPL:  37.231
Epoch-6 lr: 0.001
Epoch: 06 | Time: 0m 49s
	Train Loss: 3.000 | Train PPL:  20.076
	 Val. Loss: 3.632 |  Val. PPL:  37.777
Epoch-7 lr: 0.001
Epoch: 07 | Time: 0m 50s
	Train Loss: 2.891 | Train PPL:  18.010
	 Val. Loss: 3.539 |  Val. PPL:  34.435
Epoch-8 lr: 0.001
Epoch: 08 | Time: 0m 50s
	Train Loss: 2.765 | Train PPL:  15.887
	 Val. Loss: 3.553 |  Val. PPL:  34.903
Epoch-9 lr: 0.00

## optimizer: SGD(model.parameters(), lr = 0.7, momentum=0) + halving lr

In [64]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.859 | Test PPL: 128.898 |


## optimizer: Adam()

In [117]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.319 | Test PPL:  27.634 |


## inference

In [123]:
def translate_sentence(model, sentence, src_field, trg_field, device, max_length=50):
    

    if type(sentence) == str:
        tokens = tokenize_en(sentence)
    else:
        tokens = [token.lower() for token in sentence]
   
    tokens.insert(0, src_field.init_token)
    tokens.append(src_field.eos_token)
   
    text_to_indices = [src_field.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)
   
    # Build encoder hidden, cell state
    with torch.no_grad():
        output, hidden, cell = model.encoder(sentence_tensor)

    outputs = [trg_field.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == trg_field.vocab.stoi["<eos>"]:
            break

    translated_sentence = [trg_field.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:-1]

prediction 한국어로 번여하면 나는 환자다..?로 구글 번역기가 말해줌.. (이름같은 고유명사는 당연히 힘들듯)

In [124]:
sentence = 'My name is Karlie.'
translate_sentence(model, sentence, SRC, TRG, device, max_length=50)

['es', 'ist', 'einen', 'patienten', '.']

prediction 한국어로 번역하면 "오렌지를 든 남자. 무언가를 응시한다" 라고 구글번역기가 말해줌. 이건 좀 정확해보임

In [125]:
sentence = 'a man in an orange hat starring at something.'
translate_sentence(model, sentence, SRC, TRG, device, max_length=50)

['ein',
 'mann',
 'mit',
 'einem',
 'orangefarbenen',
 'hut',
 'starrt',
 'auf',
 'etwas',
 '.']

## BLEU score test

In [99]:
from torchtext.data.metrics import bleu_score
from tqdm import tqdm

In [140]:
label = []
predict = []
for i in tqdm(range(len(test_data.examples))):
    sentence = vars(test_data.examples[i])['src']
    label.append([vars(test_data.examples[i])['trg']])
    predict.append(translate_sentence(model, sentence, SRC, TRG, device, max_length=50))
print('finish!')
print(f"BLEU score: {bleu_score(predict, label) * 100: .2f} %")

100%|██████████| 1000/1000 [00:08<00:00, 113.39it/s]


finish!
BLEU score:  20.39 %


In [129]:
cnt = 0
for a, b in zip(label,predict):
    print(a)
    print(b)
    cnt += 1
    print()
    if cnt == 2:
        break

['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.']
['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', 'starrt', 'auf', 'etwas', '.']

['ein', 'boston', 'terrier', 'läuft', 'über', 'saftig-grünes', 'gras', 'vor', 'einem', 'weißen', 'zaun', '.']
['ein', 'motocrossfahrer', 'rennt', 'mit', 'einem', 'roten', 'spielzeug', 'auf', 'einem', 'feld', '.', '.']



In [137]:
aa = [label[0]]
bb = [[predict[0]]]
print(aa)
print(bb)

[['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.']]
[[['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', 'starrt', 'auf', 'etwas', '.']]]


In [138]:
bleu_score(aa, bb)

0.496168315410614